# This notebook helps manage SAS Model publish Destinations. Handles Private Docker destinations.

#### This cell sets default values 

In [ ]:
import sys
sys.path.append('..')
import mmAuthorization
import getpass

import requests
import json, os, pprint
import base64


### Following defines few methods and config values for later resuse

In [ ]:
def list_destinations(destination_url, auth_token): 
    headers = {
        mmAuthorization.AUTHORIZATION_HEADER: mmAuthorization.AUTHORIZATION_TOKEN + auth_token
    }

    print("List the destinations...")
    try:
        response = requests.get(destination_url, headers=headers,verify=False)
        jsondata = response.json()
        destinations = jsondata['items']
        if len(destinations) > 0:
            for destination in destinations:            
                print(destination["id"])
                print(destination["name"])
                print("===========")

    except:
        raise RuntimeError("ERROR: Could not get a destination list.")

public_ip = "PUBLIC_IP"
host_name = "fsbuwlm.fsbudev-openstack-k8s.unx.sas.com"
port = "PORT"

host_url="https://" + host_name 
destination_url = host_url + "/modelPublish/destinations/"
modelrepo_url = host_url + "/modelRepository/models/"
publishmodel_url = host_url + "/modelPublish/models"
domains_url = host_url + "/credentials/domains"
print(host_url)

### Following gets Auth token

In [ ]:
mm_auth = mmAuthorization.mmAuthorization("myAuth")

admin_userId = 'whoami'
user_passwd = getpass.getpass()

admin_auth_token = mm_auth.get_auth_token(host_url, admin_userId, user_passwd)

credential_admin_headers = {
    mmAuthorization.AUTHORIZATION_HEADER: mmAuthorization.AUTHORIZATION_TOKEN + admin_auth_token
}

credential_domain_headers = {
    "If-Match":"false",
    "Content-Type":"application/json",
    mmAuthorization.AUTHORIZATION_HEADER: mmAuthorization.AUTHORIZATION_TOKEN + admin_auth_token
}

credential_user_headers = {
    "If-Match":"false",
    "Content-Type":"application/json",
    mmAuthorization.AUTHORIZATION_HEADER: mmAuthorization.AUTHORIZATION_TOKEN + admin_auth_token
}

destination_harbor_headers = {
    "If-Match":"false",
    "Content-Type":"application/vnd.sas.models.publishing.destination.privatedocker+json",
    mmAuthorization.AUTHORIZATION_HEADER: mmAuthorization.AUTHORIZATION_TOKEN + admin_auth_token
}

print(admin_auth_token)

In [ ]:
##### create Domain

domain_name = "fsbu_domain_1"
description = 'fsbu domain 1'

my_domain_url = domains_url + "/" + domain_name
domain_attrs = {
    "id":domain_name,
    "type":"base64",
    "description": description
}

domain = requests.put(my_domain_url, 
                       data=json.dumps(domain_attrs), headers=credential_domain_headers, verify=False)

print(domain)
pprint.pprint(domain.json())

In [ ]:
### Create credential 
####
user_credential_name = admin_userId
my_credential_url = my_domain_url + "/users/" + user_credential_name

userId = "fsbu_modeluser"
password = "fsbu_modeluser"
encoded_userId = str(base64.b64encode(userId.encode("utf-8")), "utf-8")
encoded_password = str(base64.b64encode(password.encode("utf-8")), "utf-8")
credential_attrs = {
    "domainId":domain_name,
    "identityType":"user",
    "identityId":user_credential_name,
    "domainType":"base64",
    "properties":{"dockerRegistryUserId":encoded_userId},
    "secrets":{"dockerRegistryPasswd":encoded_password}
}

#credential_attrs = {
#    "domainId":domain_name,
#    "identityType":"user",
#    "identityId":user_credential_name,
#    "domainType":"base64"
#}

credential = requests.put(my_credential_url, 
                       data=json.dumps(credential_attrs), headers=credential_user_headers,verify=False)

print(credential)
pprint.pprint(credential.json())

In [ ]:
# Creates a new destination, expecting a response code of 201.

dest_name = "fsbu_dest_docker_1"
domainName = "fsbu_domain_1"
baseRepoUrl = "docker-repo.company.com:5003"
# no need of docker host in 1.1.4 since we have kaniko. 

destination_attrs = {
    "name":dest_name,
    "destinationType":"privateDocker",
     "properties": [{"name": "credDomainId",
                    "value": domainName},
                    {"name": "baseRepoUrl",
                    "value": baseRepoUrl}
                   ]
}

destination = requests.post(destination_url, data=json.dumps(destination_attrs), headers=destination_harbor_headers, verify=False)

print(destination)

In [ ]:
list_destinations(destination_url, admin_auth_token)

In [ ]:
deletedURL = destination_url + dest_name

destination = requests.delete(deletedURL, headers=credential_admin_headers)

print(deletedURL)
print(destination)
pprint.pprint(destination.json())